# 决策树经典算法及部分数学

### 信息熵

 $$ H(x) = -\sum_{i=1}^{n} {p_i\log{p_i}} \qquad Entropy(t) = -\sum_{i=0}^{c-1} {P(i|t)\log_2p(i|t)}$$

其实上面两个式子相同，都是每个事件发生的概率乘对应的信息熵大小的累加，**熵就是系统信息量的期望** $ \quad H(P):=E(P_f)$

### 相对熵(KL散度)、交叉熵
相对熵是一种量化两种概率分布P和Q之间差异的方式

$$\begin{align} D_{KL}(P||Q) &:= \sum_{i=1}^{m} {p_i\left(f_Q(q_i)-f_P(q_i)\right)}\\&=\sum_{i=1}^{m} {p_i\left(-\log_2q_i - (-\log_2p_i)\right)}\\&=\sum_{i=1}^{m} {p_i(-\log_2q_i)} - \sum_{i=1}^{m} {p_i(-\log_2p_i)}
\end{align}$$

上式可知，后面的部分 $\sum_{i=1}^{m} {p_i(-\log_2p_i)}$ 是系统 $P$ 的熵，而根据吉布斯不等式，KL散度肯定是大于等于零的

前面的部分就是交叉熵 $H(P,Q) = \sum_{i=1}^{m} {p_i(-\log_2q_i)}$ 

因为想让两个分布的熵相同，则转化为求 $H(P,Q)$ 的最小值，这个式子本身就可以作为损失函数

### ID3算法
ID3 以信息熵为基础，采用信息增益作为选择最优的分裂属性的方法，选择熵作为衡量节点纯度的标准

由上面公式可知，熵一定是大于零的，且**信息熵越大，不确定性越高，纯度越低。** 集合中的所有样本均匀混合时，信息熵最大，纯度最低。

信息增益指的就是划分可以带来纯度的提高，信息熵的下降。它的计算公式，是父亲节点的信息熵减去所有子节点的信息熵。

$$ Gain(D,a) = Entropy(D) - \sum_{i=1}^{k} {\frac{\vert D_i\vert}{\vert D\vert}Entropy(D_i)} $$

其中， 后面一项被称为条件信息熵 $H(D|A)\quad\frac{\vert D_i\vert}{\vert D\vert}$ 是每个子节点的权重，$a$ 是每个属性，$D$ 是父节点，$D_i$ 是子节点

ID3算法的划分策略：
$$ a_* = \mathop{\mathrm{argmax}}_{\mathbf{a \in A}}Gain(D,a) $$

### 优势与不足

#### 优点
- 假设空间包含所有的决策树，搜索空间完整。

#### 缺点
- 只能处理离散型属性，无法处理连续型变量例如身高体重的等
- ID3算法对于缺失值没有进行考虑
- 没有考虑过拟合的问题
- 以信息增益作为评价标准导致倾向于选择取值较多的属性

### C4.5算法
针对ID3 算法倾向于选择取值比较多的属性：c4.5采用信息增益率的方式来选择属性。

增益率定义：
$$ Gain\_ratio(D,a) = \frac{Dain(D,a)}{IV(a)} $$
其中
$$ IV(a) = -\sum_{v=1}^{V} {\frac{\lvert D^v\rvert}{\lvert D\rvert} \log_2{\frac{\lvert D^v\rvert}{\lvert D\rvert}}} $$

$IV(a)$ 被成为属性a的固有值，信息增益率对可取值较少的特征有所偏好（分母越小，整体越大）,因此，C4.5采用启发式方法，先找到信息增益旅高于平均值的特征，再从中选最高的

**思想：**
- 引入悲观剪枝策略进行后剪枝
- 引入信息增益率作为划分标准
- 将连续特征离散化，假设 n 个样本的连续特征 A 有 m 个取值，C4.5 将其排序并取相邻两样本值的平均数共 m-1 个划分点，分别计算以该划分点作为二元分类点时的信息增益，并选择信息增益最大的点作为该连续特征的二元离散分类点
- 对于缺失值：
 * 在特征值缺失的情况下进行划分特征的选择？（即如何计算特征的信息增益率）
 * 对于具有缺失值特征，用没有缺失的样本子集所占比重来折算
 * 选定该划分特征，对于缺失该特征值的样本如何处理？（即到底把这个样本划分到哪个结点里）
 * 将样本同时划分到所有子节点，不过要调整样本的权重值，其实也就是以不同概率划分到不同节点中。

**后剪枝(悲观剪枝方法)：** 欠拟合风险小，时间长

用递归的方式从低往上针对每一个非叶子节点，评估**用一个最佳叶子节点去代替这课子树是否有益。** 如果剪枝后与剪枝前相比其错误率是保持或者下降，则这棵子树就可以被替换掉。C4.5 通过训练数据集上的错误分类数量来估算未知样本上的错误率

#### 缺点
- 剪枝策略可以再优化
- C4.5 用的是多叉树，用二叉树效率更高
- C4.5 只能用于分类，不能用于回归
- C4.5 使用的熵模型拥有大量耗时的对数运算，连续值还有排序运算